In [1]:
# This is the test script for the final submission 
# No external library other than numpy is allowed 

# sys is only used to point to the correct folder
import sys
sys.path.insert(0, 'project1/scripts')

# import all the functions from the helper module 
from proj1_helpers import *
# numpy has been included in the proj1_helpers module
# Use np directly 

In [4]:
# sub_sample: take only first 50 data points 
yb, input_data, ids = load_csv_data("train.csv", sub_sample=False)


In [3]:
# yb: class label (s=1, b=-1)
print("The dimension of yb is", yb.shape)
print(yb)
# Features 
print("The dimension of input data is", input_data.shape)
print(input_data)
# ids: event ids  
print("The dimension of ids is", ids.shape)
print(ids)

The dimension of yb is (250000,)
[ 1. -1. -1. ...  1. -1. -1.]
The dimension of input data is (250000, 30)
[[ 138.47    51.655   97.827 ...    1.24    -2.475  113.497]
 [ 160.937   68.768  103.235 ... -999.    -999.      46.226]
 [-999.     162.172  125.953 ... -999.    -999.      44.251]
 ...
 [ 105.457   60.526   75.839 ... -999.    -999.      41.992]
 [  94.951   19.362   68.812 ... -999.    -999.       0.   ]
 [-999.      72.756   70.831 ... -999.    -999.       0.   ]]
The dimension of ids is (250000,)
[100000 100001 100002 ... 349997 349998 349999]
